# Градиентный бустинг

В задаче машинного обучения нам нужно оптимизировать функцию потерь $\sum_{i=1}^n L(a(x_i), y_i)$, где $L(\hat{y}, y)$ – функция потерь на одном объекте.

Предположим, что алгоритм $a$ это линейная коминация других алгоритмов: $a(x) = \sum_{t=1}^T a_t(x)$.

Зададимся вопросом: а что если мы хотим добавить ещё один алгоритм в эту комбинацию, но не просто добавить, а как можно оптимальнее с точки зрения исходной оптимизационной задачи. Тоесть уже есть какой-то алгоритм $a(x)$ и мы хотим прибавить к нему алгоритм $a_{T+1}(x)$:

$$\sum_{i=1}^n L(a(x_i) + a_{T+1}(x_i), y_i) \to \min_{a_{T+1}}$$

Допустим, что $a_{T+1}(x_i)$ достаточно малы, тогда можно приблизить исходную формулу, разложив еёв ряд Тейлора до первого члена:


$\frac{\partial{L}(a(x_i), y_i))}{\partial{a}}$

$$\sum_{i=1}^n L(a(x_i), y_i) + \sum_{i=1}^n \frac{\partial{L}}{\partial{\hat{y}}}(a(x_i), y_i)) \cdot a_{T+1}(x_i) \to \min_{a_{T+1}}$$

Первое слагаемое от $a_{T+1}$ не зависит, поэтому получается:

$$\sum_{i=1}^n \frac{\partial{L}}{\partial{\hat{y}}}(a(x_i), y_i)) \cdot a_{T+1}(x_i) \to \min_{a_{T+1}}$$

Чтобы минимизировать данное выражение требуется, чтобы вектор $\{a_{T+1}(x_i)\}_{i=1}^n$ был максимально похож на вектор $\{- \frac{\partial{L}}{\partial{\hat{y}}}(a(x_i), y_i))\}_{i=1}^n$, то есть антиградиент фунции $L(\hat{y}, y)$ на выборке.

Если $a_{T+1}$ это обучаемый алгоритм, то мы можем обучить его по $x_i$ предсказывать не исходный таргет, но $- \frac{\partial{L}}{\partial{\hat{y}}}(a(x_i), y_i))$, тем самым алгоритм будет похож на антиградиент по построению.

Таким образом, можно подобрать неплохое улучшение текущего алгоритма $a(x)$, а потом ещё раз и ещё, в итоге получив комбинацию алгоритмов, которая будет минимизировать исходный функционал.

# Бустинг над решающими деревьями

Наиболее популярное семейство алгоритмов для бустинга это деревья. Рассмотрим популярные библиотеки

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [5]:
data = pd.read_csv('HR.csv')

target = 'left'
features = [c for c in data if c != target]
print(features)

X, y = data[features], data[target]

['last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']


Качество классификации решающим деревом с настройками по-умолчанию:

In [10]:
print("XGBClassifier:", cross_val_score(XGBClassifier(verbose=False), X, y).mean())

/home/tylorn/envs/dmia/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/tylorn/envs/dmia/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


XGBClassifier: 0.779121224244849


/home/tylorn/envs/dmia/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [11]:
print("CatBoostClassifier:", cross_val_score(CatBoostClassifier(verbose=False), X, y).mean())

CatBoostClassifier: 0.7775852903914116


## Опциональное задание
Поиграйтесь с основными параметрами алгоритмов, чтобы максимизировать качество